Fonction de conversion numpy vers list et vice-versa

In [ ]:
l2n = lambda l: numpy.array(l)
n2l = lambda n: list(n)

plant_result = {}

Ouverture du fichier de données

In [ ]:
try:
   with open('donnee_entree.json') as f:
     data = json.load(f)
except:
   sys.exit("Error: enable to read file")

Création de l'environnement virtuelle

In [ ]:
ren = vtk.vtkRenderer()
ren.SetBackground(1, 1, 1)
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)

# create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

In [ ]:
Création de la chambre de culture

In [ ]:
# cube = chambre de culture
xwidth = data["room"]["width"]
ywidth = data["room"]["height"]
zwidth = data["room"]["ceiling-height"]
cube = vtk.vtkCubeSource()
cube.SetXLength(xwidth)
cube.SetYLength(ywidth)
cube.SetZLength(zwidth)
cube.Update()

cubeMapper = vtk.vtkPolyDataMapper()
cubeMapper.SetInputConnection(cube.GetOutputPort())

cubeActor = vtk.vtkActor()
cubeActor.SetMapper(cubeMapper)
cubeActor.SetOrigin(0, 0, 0) #?????
cubeActor.GetProperty().SetOpacity(0.1)
cubeActor.GetProperty().SetColor(0, 0, 1)
cubeActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
cubeActor.GetProperty().SetEdgeColor(0, 1, 0)

ren.AddActor(cubeActor)

Indiquer que ce sera des surfaces d'intersections

In [ ]:
# indiquer que ce sera des surfaces interscetions 
obbSurface = vtk.vtkOBBTree()
obbSurface.SetDataSet(cube.GetOutput())
obbSurface.BuildLocator()


Créations des normales "virtuelles" pour chaque point de la surface

In [ ]:
# Create a new 'vtkPolyDataNormals' and connect to the 'earth' sphere
normalsCalcSurface = vtk.vtkPolyDataNormals()
normalsCalcSurface.SetInputConnection(cube.GetOutputPort())

# Disable normal calculation at cell vertices
normalsCalcSurface.ComputePointNormalsOff()
# Enable normal calculation at cell centers
normalsCalcSurface.ComputeCellNormalsOn()
# Disable splitting of sharp edges
normalsCalcSurface.SplittingOff()
# Disable global flipping of normal orientation
normalsCalcSurface.FlipNormalsOff()
# Enable automatic determination of correct normal orientation
normalsCalcSurface.AutoOrientNormalsOn()
# Perform calculation
normalsCalcSurface.Update()

Création de la plante 

In [ ]:
# plante
plant = vtk.vtkPlaneSource()
plant.SetOrigin(-30, -75, -10)  #premier point
plant.SetPoint1(-30, -75, 30)   #deuxième point
plant.SetPoint2(40, -75, -10)   #troisième point
plant.SetXResolution(15)
plant.SetYResolution(15)
plant.Update()

plantMapper = vtk.vtkPolyDataMapper()
plantMapper.SetInputConnection(plant.GetOutputPort())

plantActor = vtk.vtkActor()
plantActor.SetMapper(plantMapper)
plantActor.GetProperty().SetColor(0, 1, 0)
plantActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
plantActor.GetProperty().SetEdgeColor(0, 1, 0)

ren.AddActor(plantActor)

In [ ]:
# indiquer que ce sera des surfaces interscetions 
obbPlant = vtk.vtkOBBTree()
obbPlant.SetDataSet(plant.GetOutput())
obbPlant.BuildLocator()


Lampes + boucle

In [ ]:
v=0
n=0
while v < 3: # la boucle se termine à la fin du code!!
 x = data["array"][str(v)]["x"] #par rapport à l'origine qui est au centre du cube
 y = data["array"][str(v)]["y"]
 z = data["array"][str(v)]["z"]

 Lamp = vtk.vtkSphereSource()
 Lamp.SetCenter(x, y, z)
 Lamp.SetRadius(5)
 Lamp.SetStartTheta(data["array"][str(v)]["angle"])
 Lamp.SetPhiResolution(15)
 Lamp.SetThetaResolution(15)

 mapper = vtk.vtkPolyDataMapper()
 mapper.SetInputConnection(Lamp.GetOutputPort())

 pointActor = vtk.vtkActor()
 pointActor.SetMapper(mapper)
 pointActor.SetOrigin(0, 0, 0) #???
 pointActor.GetProperty().SetColor(color(data["array"][str(v)]["wavelength"]))
 pointActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
 pointActor.GetProperty().SetEdgeColor(color(data["array"][str(v)]["wavelength"]))

 ren.AddActor(pointActor)

 
 
#=============================================================================
#Intersection
#-----------------------------------------------------------------------------

# Création des vecteurs normaux des centres des meshs de la lampes
#-----------------------------------------------------------------------------
# Get the point centers from 'cellCenterCalc'

 cellCenterCalcLamp = vtk.vtkCellCenters()
 cellCenterCalcLamp.SetInputConnection(Lamp.GetOutputPort())
 cellCenterCalcLamp.Update()

 pointsCellCentersLamp = cellCenterCalcLamp.GetOutput(0)

# Create a new 'vtkPolyDataNormals' and connect to the 'lamp' half-sphere
 normalsCalcLamp = vtk.vtkPolyDataNormals()
 normalsCalcLamp.SetInputConnection(Lamp.GetOutputPort())

# Disable normal calculation at cell vertices
 normalsCalcLamp.ComputePointNormalsOff()
# Enable normal calculation at cell centers
 normalsCalcLamp.ComputeCellNormalsOn()
# Disable splitting of sharp edges
 normalsCalcLamp.SplittingOff()
# Disable global flipping of normal orientation
 normalsCalcLamp.FlipNormalsOff()
# Enable automatic determination of correct normal orientation
 normalsCalcLamp.AutoOrientNormalsOn()
# Perform calculation
 normalsCalcLamp.Update()

#-----------------------------------------------------------------------------
# RAY-TRACING
#----------------------------------------------------------------------------
  #Extract the normal-vector data at the sun's cells
 normalsLamp = normalsCalcLamp.GetOutput().GetCellData().GetNormals()
   # Extract the normal-vector data at the earth's cells
 normalsSurface = normalsCalcSurface.GetOutput().GetCellData().GetNormals()

 # points_lampes_surfaces.update({'lampe'+str(v):{'coord': {}, 'vecteur_incident': {},'vecteur_reflechi':{}, 'point':{}}})

 for idx in range(pointsCellCentersLamp.GetNumberOfPoints()):
     
#-----------Premier lancer de rays--------------------------------------------
    
    # Get coordinates of lamp's cell center
    pointLamp = pointsCellCentersLamp.GetPoint(idx)
    # Get normal vector at that cell
    normalLamp = normalsLamp.GetTuple(idx)
    
    # Calculate the 'target' of the ray 
    pointRayTarget = n2l(l2n(pointLamp) + 500*l2n(normalLamp))
    
    # Check if there are any intersections for the given ray with surface
    if isHit(obbSurface, pointLamp, pointRayTarget):  # intersections were found
        # Retrieve coordinates of intersection points and intersected cell ids
        pointsInter, cellIdsInter = GetIntersect(obbSurface, pointLamp, pointRayTarget)
        # Render lines/rays emanating from the slamp. 
        addLine(ren, pointLamp, pointsInter[0], color(data["array"][str(v)]["wavelength"]), opacity = 0.1)
        # Render intersection points
        addPoint(ren, pointsInter[0], color(data["array"][str(v)]["wavelength"]))
        # Calculate the incident ray vector
        # Get the normal vector at the surface cell that intersected with the ray
        normalSurface = normalsSurface.GetTuple(cellIdsInter[0])
        vecInc = n2l(l2n(pointRayTarget) - l2n(pointLamp))
        # Calculate the reflected ray vector
        vecRef = calcVecR(vecInc, normalSurface)
        # Calculate the 'target' of the reflected ray 
        pointRayReflectedTarget = n2l(l2n(pointsInter[0]) + l2n(vecRef))
        
        # Fresnel 
        R, T =Fresnel(data["room"]["n_air"],data["room"]["n_paroi"],vecInc, normalSurface)
    
    # Check if there are any intersections for the given ray with plante  
    if isHit(obbPlant, pointLamp, pointRayTarget):  # intersections were found
        # Retrieve coordinates of intersection points and intersected cell ids
        ptsInter, cIdsInter = GetIntersect_plant(obbPlant, pointLamp, pointRayTarget)
        # Render lines/rays emanating from the slamp. 
        addLine(ren, pointLamp, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        # Render intersection points
        addPoint(ren, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        
        # plant_result.update({'area'+str(cIdsInter):{'cordinate': list(),'lamp': list(), 'wavelength': list(),'rayon':list()}})
        # plant_result['area'+str(cIdsInter)]['cordinate'].append(ptsInter[0])
        # plant_result['area'+str(cIdsInter)]['lamp'].append(v)
        # plant_result['area'+str(cIdsInter)]['wavelength'].append(data["array"][str(v)]["wavelength"])
        # plant_result['area'+str(cIdsInter)]['rayon'].append('direct')
        
    
        
#-------------Première réflection----------------------------------------------
      
      # Check if there are any intersections for the given ray
          # Check if there are any intersections for the given ray with plante  
    if isHit(obbSurface,  pointsInter[0], pointRayReflectedTarget):  # intersections were found
          # Retrieve coordinates of intersection points and intersected cell ids
            pointsInter, cellIdsInter = GetIntersect(obbSurface,  pointsInter[0], pointRayReflectedTarget)
     
            addLine(ren,  pointsInter[0], pointsInter[1], color(data["array"][str(v)]["wavelength"]),opacity=0.1)
            addPoint(ren, pointsInter[1], color(data["array"][str(v)]["wavelength"]))
         
           # Calculate the incident ray vector
           # Get the normal vector at the earth cell that intersected with the ray
            normalSurface = normalsSurface.GetTuple(cellIdsInter[1])
            vecInc = n2l(l2n(pointRayReflectedTarget) - l2n(pointsInter[1]))
           # Calculate the reflected ray vector
            vecRef = calcVecR(vecInc, normalSurface)
            
            pointRayReflectedTarget = n2l(l2n(pointsInter[1]) + l2n(vecRef))
            
    # Check if there are any intersections for the given ray with plante  
    if isHit(obbPlant, pointsInter[0], pointRayReflectedTarget):  # intersections were found
        # Retrieve coordinates of intersection points and intersected cell ids
        ptsInter, cIdsInter = GetIntersect_plant(obbPlant,  pointsInter[0], pointRayReflectedTarget)
        # Render lines/rays emanating from the slamp. 
        addLine(ren, pointsInter[0], ptsInter[0], color(data["array"][str(v)]["wavelength"]))
        # Render intersection points
        addPoint(ren, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
      

        # plant_result['area'+str(cIdsInter)]['cordinate'].append(ptsInter[0])
        # plant_result['area'+str(cIdsInter)]['lamp'].append(v)
        # plant_result['area'+str(cIdsInter)]['wavelength'].append(data["array"][str(v)]["wavelength"])
        # plant_result['area'+str(cIdsInter)]['rayon'].append('First reflection')
        
#---------- deuxième reflection------------------------------------------------


 v+=1


Visualisation

In [ ]:
iren.Initialize()
renWin.Render()
iren.Start()